In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
from sklearn.model_selection import train_test_split  
from IPython.display import display   ## To display function results without needing a Return

from sklearn.metrics import mutual_info_score

from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LogisticRegression

In [4]:
!pip install wget

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wget'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=d047707a35132f46e5448b57770be2ff2155d168aa8b9345d86f152759a16656
  Stored in directory: /home/codespace/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
## USE WGET TO DOWNLOAD FILE

# 2. Define the URL of the CSV file you want to download
file_url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

# 3. Define the local filename to save the downloaded file
filename = 'course_lead_scoring.csv'

# 4. Use `wget` with an exclamation mark to download the file
!wget -O {filename} {file_url}

--2025-10-14 19:54:17--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-14 19:54:18 (7.56 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [3]:
df = pd.read_csv(filename)

df.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0


# Data Preparation

In [4]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
## Replace Null values for categorical columns with 'NA'
categorical = list(df.dtypes[df.dtypes=='object'].index)

df[categorical] = df[categorical].fillna('NA')

df[categorical].isnull().sum()


lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [6]:
## Replace Null values for numerical columns with 0.0
numerical = list(df.dtypes[(df.dtypes=="int64") | (df.dtypes=="float64")].index)

df[numerical] = df[numerical].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### QUESTION 1 

What is the most frequent observation (mode) for the column industry?

In [7]:
## QUESTION 1 What is the most frequent observation (mode) for the column industry?

df.industry.mode()

0    retail
Name: industry, dtype: object

### QUESTION 2 

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [23]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
df_number = df[numerical]

df_number

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1
3,2,83843.0,1,0.87,0
4,3,85012.0,3,0.62,1
...,...,...,...,...,...
1457,1,0.0,4,0.53,1
1458,3,65259.0,2,0.24,1
1459,1,45688.0,3,0.02,1
1460,5,71016.0,0,0.25,1


In [9]:
df_number.corr().abs()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,0.023565,0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


What are the two features that have the biggest correlation?

interaction_count and lead_score  --> 0.0009888

number_of_courses_viewed and lead_score --> 0.004879

number_of_courses_viewed and interaction_count  --> 0.023565

annual_income and interaction_count  --> 0.027036

Only consider the pairs above when answering this question.  

ANSWER = The 2 highest correlations are shared between {"number_of_courses_viewed" and "interaction_count"} 

and {"annual_income" and "interaction_count"}



# QUESTION 3 Split Data into Train/Validation/Test datasets


Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value converted is not in your dataframe.

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
## set 80/20  Train/Test split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## Split Traiining set into training and validation
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [12]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [13]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [14]:
df_val.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,healthcare,3,52220.0,unemployed,europe,1,0.07
1,organic_search,technology,3,59656.0,unemployed,middle_east,4,0.65


### QUESTION 3

In [15]:
from sklearn.metrics import mutual_info_score

In [16]:
### Calculate mutual information score between 'converted' and other categorical variables

df_full_train.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1


In [17]:
## sklearn mutual info score takes a series to series comparison as its paramters
    ## so create function and .apply() the function to each column individually

def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [18]:
### apply mutual_info_score method to each filtered categorical column using defined function and .apply() 
    ## save results into new DataFrame and sort results

mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

### QUESTION 3 ANSWER = "lead_source" variable has the largest mutual information score with 'y' variable 'converted'

# QUESTION 4

In [19]:
## ONE HOT ENCODING USING SKLEARN 
    ## NOTE THAT THE VECTORIZER TAKES DICTIONARY AS ITS INPUT

from sklearn.feature_extraction import DictVectorizer

In [20]:
df_train.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77


In [21]:
#must remove 'converted' from list of numerical columns
    ## use list comprehension
numerical = [string for string in numerical if string != "converted"]
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [22]:
dv = DictVectorizer(sparse=False)
full_features = categorical + numerical
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [41]:
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
## Train Logistic Regression model with specified parameters in homework #3
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [39]:
##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)

## Check Accuracy of logistic prediction "y_pred" versus "y_val" actuals
round((y_val == converted_decision).mean(),4)

np.float64(0.6997)

# Question 5
Let's find the least useful feature using the feature elimination technique.

Train a model using the same features and parameters as in Q4 (without rounding).

Now exclude each feature from this set and train a model without it. Record the accuracy 

for each model.

For each feature, calculate the difference between the original accuracy and the 

accuracy without the feature.

Which of following feature has the smallest difference?

'industry'

'employment_status'

'lead_score'

In [46]:
## libraries and modules needed (same as in #4)
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

## create one-hot encoding dictionary (numerical columns value remains intact)
dv = DictVectorizer(sparse=False)

In [108]:
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [109]:
## Baseline model containing all features 
    ## model from Question #4 above

## Train Logistic Regression model with specified parameters in homework #3

full_features = categorical + numerical
train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)

df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val
df_pred['accuracy'] = df_pred.prediction == df_pred.actual

df_pred

,probability,prediction,actual,accuracy
0,0.611922,1,0,False
1,0.799826,1,1,True
2,0.530213,1,0,False
3,0.471315,0,0,True
4,0.570661,1,0,False
...,...,...,...,...
288,0.419342,0,0,True
289,0.710539,1,1,True
290,0.418185,0,0,True
291,0.744835,1,1,True


In [110]:
## full features dictionary

fulldict = {'full_features': df_pred.accuracy.mean()}

fulldict

{'full_features': np.float64(0.6996587030716723)}

In [80]:
full_features = categorical + numerical

full_features

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [113]:
## Iteration to train model each time while removing 1 feature and comparing Accuracy results
### REMOVE 'lead_source'
full_features = categorical + numerical

full_features.remove('lead_source')

train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)


df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = converted_decision.astype(int)
df['actual'] = y_val
df['accuracy'] = df.prediction == df.actual

df1dict = {'lead_source': df1.accuracy.mean()}

df1dict

#df1.accuracy.mean()

{'lead_source': np.float64(0.6996587030716723)}

In [114]:
## Iteration to train model each time while removing 1 feature and comparing Accuracy results
### REMOVE 'industry'
full_features = categorical + numerical

variable = 'industry'

full_features.remove(variable)

train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)


df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = converted_decision.astype(int)
df['actual'] = y_val
df['accuracy'] = df.prediction == df.actual

df2dict = {variable: df.accuracy.mean()}

df2dict

{'industry': np.float64(0.6996587030716723)}

In [97]:
full_features = categorical + numerical

full_features

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [115]:
## Iteration to train model each time while removing 1 feature and comparing Accuracy results

full_features = categorical + numerical

variable = 'employment_status'

full_features.remove(variable)

train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)


df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = converted_decision.astype(int)
df['actual'] = y_val
df['accuracy'] = df.prediction == df.actual

df3dict = {variable: df.accuracy.mean()}

df3dict

{'employment_status': np.float64(0.6962457337883959)}

In [99]:
full_features = categorical + numerical

full_features

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [116]:
## Iteration to train model each time while removing 1 feature and comparing Accuracy results

full_features = categorical + numerical

variable = 'location'

full_features.remove(variable)

train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)


df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = converted_decision.astype(int)
df['actual'] = y_val
df['accuracy'] = df.prediction == df.actual

df4dict = {variable: df.accuracy.mean()}

df4dict

{'location': np.float64(0.7098976109215017)}

In [117]:
full_features = categorical + numerical

full_features

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [118]:
## Iteration to train model each time while removing 1 feature and comparing Accuracy results

full_features = categorical + numerical

variable = 'lead_score'

full_features.remove(variable)

train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

##Calculate probability of binary outcome on Validation training set
y_pred = model.predict_proba(X_val)[:,1]

## Assign prediction for "converted" binary value
converted_decision = (y_pred >= 0.5)


df = pd.DataFrame()
df['probability'] = y_pred
df['prediction'] = converted_decision.astype(int)
df['actual'] = y_val
df['accuracy'] = df.prediction == df.actual

df5dict = {variable: df.accuracy.mean()}

df5dict

{'lead_score': np.float64(0.7064846416382252)}

In [126]:
# Merge all dictionaries into one using dictionary unpacking
merged = {**fulldict, **df1dict, **df2dict, **df3dict, **df4dict, **df5dict}

# Create a DataFrame with a single row
#df_all = pd.DataFrame([merged])
df_alldicts = pd.DataFrame([merged])

df_alldicts

,full_features,lead_source,industry,employment_status,location,lead_score
0,0.699659,0.699659,0.699659,0.696246,0.709898,0.706485


In [127]:
## Calculate difference from baseline model with full features

dfx = df_alldicts.copy()

dfx['diff_industry'] = dfx['full_features'] - dfx['industry']
dfx['diff_employment'] = dfx['full_features'] - dfx['employment_status']
dfx['diff_lead'] = dfx['full_features']- dfx['lead_score']

dfx

,full_features,lead_source,industry,employment_status,location,lead_score,diff_industry,diff_employment,diff_lead
0,0.699659,0.699659,0.699659,0.696246,0.709898,0.706485,0.0,0.003413,-0.006826


### QUESTION 5 ANSWER = Industry has the smallest differnce in accuracy when compared to the baseline model containing all features

# QUESTION 6


Now let's train a regularized logistic regression.

Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].

Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

0.01

0.1

1

10

100


In [129]:
### Use Full_features 
    ## all categorical and all numerical features

## pass all features to dictionary and then fit to Vectorizer model for array creation of features
    ## fit these features to train your logistic regression model

full_features = categorical + numerical
train_dict = df_train[full_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[full_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [132]:
# Regularized C values
C_values = [0.01, 0.1, 1, 10, 100]

results = []

for c in C_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)


    acc = round((y_val == y_pred).mean(),8)

    
    results.append({'C': c, 'Accuracy': acc})

df_results = pd.DataFrame(results)
print(df_results)

        C  Accuracy
0    0.01  0.699659
1    0.10  0.699659
2    1.00  0.699659
3   10.00  0.699659
4  100.00  0.699659
